In [30]:
!pip install python-dateutil
!pip install mechanize
!pip install pandas_datareader
!pip install yahoo_finance_api2
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

SyntaxError: ignored

In [ ]:

import sys
ROOT_PATH = '/content/drive/MyDrive/finance'
sys.path.append(ROOT_PATH)


from pandas_datareader import data
from datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import talib as ta
import warnings
warnings.simplefilter('ignore')


import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import talib
import SBIcomm

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [18]:
sbi=SBIcomm.SBIcomm("342-042-0476","sasabe170")
sbi.get_value("6758")

Traceback (most recent call last):
  File "/content/drive/MyDrive/finance/SBIcomm.py", line 287, in get_value
    end_price = float(doc.xpath(self._x("/tr[2]/td/font/font"))[0].text.replace(",", ""))
IndexError: list index out of range

Cannot Get Value! 6758


(datetime.date(2021, 3, 20), None)

In [ ]:
stock_list=["QQQ"]

In [89]:
def stock_data(stock_code,frequency=1,weekly=True):
    my_share = share.Share(stock_code)
    symbol_data = None
    
    if weekly == True:
        symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,
                                              1,
                                              share.FREQUENCY_TYPE_DAY,
                                             1)
        
    elif weekly == False:
        symbol_data = my_share.get_historical(share.PERIOD_TYPE_DAY,
                                              2,
                                              share.FREQUENCY_TYPE_HOUR,
                                             1)

    df = pd.DataFrame(symbol_data)
    date_agg=lambda x:datetime.datetime.utcfromtimestamp(x/1000)
    df['timestamp'] = df['timestamp'].apply(date_agg)
    if weekly ==True:
        to_day=lambda x:datetime.datetime.strftime(x,'%A')
        df["day"]=df['timestamp'].apply(to_day)
        df_new=pd.DataFrame()
        for i in range(len(df.loc[df["day"]=="Tuesday"])):
            if df.iloc[df.loc[df["day"]=="Tuesday"].index[i]-1]["day"]=="Monday":
                df_=df.iloc[df.loc[df["day"]=="Tuesday"].index[i]-1]
                df_new=pd.concat([df_new,df_],axis=1)
            else:
                df_=df.iloc[df.loc[df["day"]=="Tuesday"].index[i]]
                df_new=pd.concat([df_new,df_],axis=1)
        df_new=df_new.T
    else:
        df_new=df
    
    
    return df_new

def get_statistics(df):
  df = df.loc[df['volume']!=0]
  close = df['close']
  df['macd'],df['macdsignal'],df['macdhist'] = ta.MACD(close,fastperiod=12,slowperiod=26,signalperiod=9)
  df['RSI'] = ta.RSI(close,timeperiod=14)
  return df

In [ ]:
for stock in stock_list:
  df=stock_data(i,frequency=1,weekly=False)
  get_statistics(df).iloc[-1]

In [111]:
!python /content/drive/MyDrive/finance/mail_sending.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/finance/mail_sending.py", line 55, in <module>
    main()
  File "/content/drive/MyDrive/finance/mail_sending.py", line 41, in main
    creds = flow.run_local_server()
  File "/usr/local/lib/python3.7/dist-packages/google_auth_oauthlib/flow.py", line 458, in run_local_server
    host, port, wsgi_app, handler_class=_WSGIRequestHandler
  File "/usr/lib/python3.7/wsgiref/simple_server.py", line 153, in make_server
    server = server_class((host, port), handler_class)
  File "/usr/lib/python3.7/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/usr/lib/python3.7/wsgiref/simple_server.py", line 50, in server_bind
    HTTPServer.server_bind(self)
  File "/usr/lib/python3.7/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.7/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already i

In [90]:
#gmail apiに関して
client_api="492916687908-elj1ubt559ser41t9s0hitt2cq6hhi4e.apps.googleusercontent.com"
client_server="SGAIRK7fCOh3gwPxkdD8pNhW"

In [21]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64
from email.mime.text import MIMEText
from apiclient import errors
# 1. Gmail APIのスコープを設定
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
# 2. メール本文の作成
def create_message(sender, to, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    encode_message = base64.urlsafe_b64encode(message.as_bytes())
    return {'raw': encode_message.decode()}
# 3. メール送信の実行
def send_message(service, user_id, message):
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print('Message Id: %s' % message['id'])
        return message
    except errors.HttpError as error:
        print('An error occurred: %s' % error)
# 4. メインとなる処理
def main(message):
    # 5. アクセストークンの取得
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    service = build('gmail', 'v1', credentials=creds)
    # 6. メール本文の作成
    sender = 's.hikaru.yakyu.0823@gmail.com'
    to = ['s.hikaru.yakyu.0823@gmail.com',"ring1289@keio.up","rid.7sixers6@gmail.com"]
    subject = 'メール送信自動化テスト'
    message_text = 'message'
    message = create_message(sender, to, subject, message_text)
    # 7. Gmail APIを呼び出してメール送信
    send_message(service, 'me', message)
  
main("test")


AttributeError: ignored

In [12]:
!pwd

/c/Users/shika


In [88]:
import datetime
df=stock_data("ACN",frequency=1,weekly=False)
get_statistics(df).iloc[-1]

timestamp     2021-03-19 19:30:00
open                       266.61
high                      267.115
low                        263.55
close                      263.65
volume                     803101
macd                     0.066286
macdsignal                0.16686
macdhist                -0.100574
RSI                       41.6723
Name: 177, dtype: object

In [81]:
df = df.loc[df['volume']!=0]
code="ACN"#ETF[0]
date = df['timestamp']
close = df['close']

df['macd'],df['macdsignal'],df['macdhist'] = ta.MACD(close,fastperiod=12,slowperiod=26,signalperiod=9)
df['RSI'] = ta.RSI(close,timeperiod=14)
#移動平均なので、最初のspan日だけ欠損値になる。
#それを防ぐために、ループを掛ける前に修正を行う
rsi_list=df["RSI"][df["RSI"].notna()]
macd_list,signal_list=df['macd'][df["RSI"].notna()],df['macdsignal'][df["RSI"].notna()]
close_revised=close[df["RSI"].notna()]



In [82]:
def get_statistics(df):
  close = df['close']
  df['macd'],df['macdsignal'],df['macdhist'] = ta.MACD(close,fastperiod=12,slowperiod=26,signalperiod=9)
  df['RSI'] = ta.RSI(close,timeperiod=14)
  return df

,timestamp,open,high,low,close,volume,macd,macdsignal,macdhist,RSI
2,2021-03-10 14:30:00,256.649994,257.940002,255.600006,257.000000,241516,NaN,NaN,NaN,NaN
3,2021-03-10 15:00:00,256.980011,257.209991,254.860001,254.955002,121890,NaN,NaN,NaN,NaN
4,2021-03-10 15:30:00,255.000000,255.830002,254.610001,255.580002,116528,NaN,NaN,NaN,NaN
5,2021-03-10 16:00:00,255.679993,256.459991,255.570007,256.290009,120131,NaN,NaN,NaN,NaN
6,2021-03-10 16:30:00,256.339996,256.410004,255.690002,256.035004,90789,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
173,2021-03-19 17:30:00,266.010010,266.690002,265.790009,266.489197,87662,0.054025,0.127065,-0.073040,53.547674
174,2021-03-19 18:00:00,266.489990,267.320007,266.329987,267.209991,103232,0.166885,0.135029,0.031856,56.312872
175,2021-03-19 18:30:00,267.239990,267.619995,266.850006,267.619995,123658,0.286112,0.165246,0.120867,57.849888
176,2021-03-19 19:00:00,267.670013,267.709015,266.140015,266.651886,143317,0.299036,0.192004,0.107032,53.099425
